# Geophysical Inversion

This application provides an interface to geophysical inversion using the [SimPEG](https://simpeg.xyz/) open-source algorithms. The application currently supports

 - Electromagnetic (time or frequency) data using a Laterally Constrained 1D approach
 - Gravity and magnetics (field and/or tensor) data using an octree mesh tiling approach.
         

  <img align="right" width="50%" src="./images/Geophysical_inversion_app.png">
  

New user? Visit the [Getting Started](../installation.rst) page.

---
**NOTE**
 

For 3D gravity and magnetic vector inversion, it is recommended to set your  from a solid-state drive (SSD), as sensitivities are stored in chunks and accessed in parallel using the [Dask](https://dask.org/) library.

---

## Application
The following sections provide details on the different parameters controling the application. Interactive widgets shown below are for demonstration purposes only.

In [1]:
from geoapps.inversion import InversionApp

app = InversionApp(h5file=r"../../../assets/FlinFlon.geoh5")
app.main

## Project Selection

Select and connect to an existing **geoh5** project file containing data. 

In [2]:
app.project_panel

See the [Project Panel](base_application.ipynb#Project-Panel) page for more details.

## Object/Data Selection

List of objects available containing geophysical data to be inverted. Currently supports objects of type `Points`, `Curve`, `Surface` and `Grid2D`. Multiple data channels and/or data groups can be selected. 

In [3]:
app.data_panel

## Window Selection

Manual selection of an area of interest and data resolution. Positive (solid) and negative (dash) contour lines are traced immediately after changes made to the contour values.

In [4]:
app.window_selection

See the [Map View Selection](view_selection.ipynb#Map-View-Selection) page for more details.

## Data Channel Options

Define geophysical data.

### Survey Type

List of available survey types.

In [5]:
app.system

Dropdown(description='Survey Type: ', options=('Magnetics', 'Gravity', 'AeroTEM (2007)', 'AeroTEM (2010)', 'DI…

The application will attempt to assign the correct system based on
known data field name (e.g. *CPI* => *DIGHEM*).

#### Inducing field parameters

For magnetic data only, set the inducing field parameters:

In [6]:
app.inducing_field


Text(value='60000, 79, 11', description='Inducing Field [Amp, Inc, Dec]', style=DescriptionStyle(description_w…

### Channel Options

Set parameters to invividual [Data](#Object/Data-Selection) channels.

In [7]:
app.data_channel_choices.data_channel_options['tmi']

#### Active

Checked if the channel is to be used in the inversion

#### Data channel:

Association between the "data channel" and one of the expected "system channel".

#### Error (%, floor)

Estimated data uncertainties associated with the data channel, expressed as a
percent (decimal) and floor value. 

As example for magnetic data: 0.1, 1 => $0.1 * |d_{tmi}| + 1\;nT$

#### Offsets (EM systems)

Determine the constant offsets, in meters, between the receiver and the transmitter center location. 

Used for the numerical forward calculations. 

## Topography, Sensor and Line Options

In [8]:
app.spatial_choices

Dropdown(options=('Topography', 'Sensor'), value='Topography')

### Topography

Set the air/ground interface of the inversion model.

In [9]:
app.topography.main

#### Object

Set the topography based on an object stored in the target `geoh5` project. The Z value of the cells/vertices can be assigned based on a chosen `Data` field.

#### Relative to Sensor

Topography is defined by a fixed vertical offset from a selected object position (vertices or centroids), also referred to as a "draped height".

#### Constant

Topography is defined as a flat surface with constant elevation.

### Sensor

Defines the sensors position in 3D space.

In [10]:
app.sensor.main

#### Sensor Location

Sensor position defined by a constant offset from the vertices of the selected
object. Typically used for towed system where the GPS receiver is on the
aircraft.

#### Topo and radar

Receiver locations defined by the vertices horizontal positions [x, y] and z value
interpolated from topography + clearance height. Typically used for gridded
data with constant draped height or for airborne survey with inaccurate GPS
elevation (radar height).

### Line ID

**(EM only)**: Line channel and line number to be inverted.

In [11]:
app.lines.main

## Inversion Options

List of parameters controlling the inversion.

In [12]:
app.inversion_parameters.main

### Output Name

Name given to the inversion group added to the ANALYST project.

### Target Misfit

Target data misfit where 1 = number of data

In [13]:
app.inversion_parameters.chi_factor

FloatText(value=1.0, description='Target misfit')

### Uncertainty mode

(EM Only) Global change to the data uncertainties 

In [14]:
app.inversion_parameters.uncert_mode

RadioButtons(index=1, options=('Estimated (%|data| + background)', 'User input (%|data| + floor)'), value='Use…

**Estimated** Floor uncertainties re-assgined based on the best-fitting halfspace model. Useful when dealing with fast time domain decays due to IP effects.

**User Input** (Default) Keep the same uncertainties as defined in the [Channel Options](#Channel-Options) 

### Starting Model

Initial model used to begin the inversion.


In [15]:
app.inversion_parameters.starting_model.main

#### Model

Model object and values selected from any `Points`, `Curve`, `Surface`, `BlockModel` or `Octree` object.
Values are interpolated onto the inversion mesh using a nearest neighbor approach.

#### Constant

Constant background half-space value.

### Susceptibility model 

**(FEM Only)**

Susceptibility values used in the forward calculations only.

In [16]:
app.inversion_parameters.susceptibility_model.main

#### None

No susceptibility model used.

#### Model

Susceptibitlity model selected from any `Points`, `Curve`, `Surface`, `BlockModel` or `Octree` object.
Values are interpolated onto the inversion mesh using a nearest neighbor approach.

#### Constant

Constant susceptibility half-space value.

### Regularization

See the [Regularization Panel](#Regularization-Panel) section.

### Upper-Lower Bounds

Upper and lower bound constraints applied on physical property model.

In [17]:
app.inversion_parameters.bound_panel

If left empty, the inversion uses [$-\infty$, $\infty$] physical property bounds.

### Mesh

See the [Mesh Design](#Mesh-Design) section.

### Ignore Values

#### Data

Flag value ignored by the inversion by assigning $\infty$ uncertainties on the data points.

In [18]:
app.inversion_parameters.ignore_values

Text(value='-99999', description='Data (i.e. <0 = no negatives)')

#### Air cells

Value assigned to the output model cells above topography. Defaults to $10^{-8}$ S/m for EM inversions

In [19]:
app.inversion_parameters.air_values

FloatText(value=0.0, description='Air cells fill value')

### Optimization

Parameters controlling various aspects of the projected Gauss-Newton inversion algorithm.

In [20]:
app.inversion_parameters.optimization

#### Max beta Iterations

Maximum number of $\beta$-iterations allowed.
Note that when applying sparse norms, the inversion may require $>20$ iterations to converge.


#### Target misfit

Target data misfit where $\chi=1$ corresponds to $\phi_d=N$ (number of data). (See [Data Misfit and Uncertainties](https://giftoolscookbook.readthedocs.io/en/latest/content/fundamentals/Uncertainties.html#data-misfit-and-uncertainties))


#### Starting trade-off ($\beta$)

**ratio**: Factor multiplying the initial $\beta$ defined by the ratio between the initial misfit and regularization:

\begin{equation}
\beta_0 = \gamma * \frac{\phi_d}{ \phi_m}
\end{equation}

**value**: Fixed $\beta$ value specified by the user.

#### Max CG Iterations

Maximum number of Conjugate Gradient (CG) iterations per Gauss-Newton solve.


#### CG Tolerance

Threshold on the minimum Conjugate Gradient (CG) step to end the Gauss-Newton solve.


#### Max CPUs

Maximum number of threads used for the parallelization. Defaults to half the system thread count.


#### Max RAM (Gb)

Approximate memory (RAM) used during the inversion. The value is used to calculate the chunk size for the storage of the sensitivity matrix on solid-state drive.
(See [dask.array.to_zarr](https://docs.dask.org/en/latest/array-api.html#dask.array.to_zarr))


## Regularization Panel

Parameters controlling the regularization function.

### Reference model

Reference model values used to contrain the inversion.

In [21]:
app.inversion_parameters.reference_model.main

#### None

No reference model used. This is equivalent as setting the [Alphas](#Alpha-Parameters) `s` parameter to zero.

#### Model

Reference model selected from any `Points`, `Curve`, `Surface`, `BlockModel` or `Octree` object.
Values are interpolated onto the inversion mesh using a nearest neighbor approach.

#### Constant

Constant reference half-space value.

#### Best-fitting halfspace

**(EM only)** Run a preliminary inversion to determine a best-fitting halfspace
(single conductivity value) at each data station.

### Alpha Parameters

Scaling between the components of the regularization function.

In [22]:
app.inversion_parameters.alphas


Text(value='1.0, 1.0, 1.0, 1.0', description='Scaling alpha_(s, x, y, z)')

See the [SimPEG.API](https://docs.simpeg.xyz/content/api_core/api_Regularization.html#SimPEG.regularization.Simple) for technical details.

### $l_p$-norms

Approximated norms applied to the components of the regularization.

In [23]:
app.inversion_parameters.norms


Text(value='0, 2, 2, 2', continuous_update=False, description='Norms p_(s, x, y, z)', style=DescriptionStyle(d…

See notes on [Sparse and Blocky Norms](https://giftoolscookbook.readthedocs.io/en/latest/content/fundamentals/Norms.html#sparse-and-blocky-norms) for technical details.

## Mesh Design

Definition of the grid containing the physical property model.

### Octree Mesh

**(Gravity and magnetics)**

Parameters used for the construction of a global `Octree` mesh.

In [24]:
app.inversion_parameters.mesh.main

#### Smallest cells

Dimensions (x,y,z) of the smallest octree cells.


####  Layers below topography


Number of layers of cells at each octree level below the topography surface.
As topography generally extends further, it is common practice to use only
coarse cells once outside of the survey area.


####  Layers below data

Number of layers of cells at each octree level below the observation points.


####  Minimum depth


Minimum depth (m) of the mesh, rounded up to the next power of 2.


####  Padding [W,E,N,S,D,U]

Additional padding distance (m) along West, East, North, South, Down and Up,
chosen by default to be half of the data extent in each direction. Additional
padding can be added at depth, but it is recommended to set it through the
[Minimum depth](#Minimum-depth) parameter.


#### Max triangulation length


Parameter controlling the Delaunay triangulation used for the refinement of
the mesh around topography and observation points. Large triangles generated
between points far apart are filtered out, reducing the number of small cells
in regions without data.

 <img align="center" width="50%" src="./images/Octree_refinement.png">

### 1D Mesh

**(EM 1D only)**

For the laterally constrained 1D inversion (time or frequency), the earth is
discretized into layer models connected by the regularization function. The
following parameters define the 1D mesh repeated below each transmitter/receiver pairs

In [25]:
app.mesh_1D.main

#### Smallest cell (m)

Smallest layer at the top of the 1D mesh.


#### Expansion factor

Rate of cell expansion from the smallest cell size.


#### Number of cells

Total number of layers below each station.


#### Max depth

Returns the full depth of the 1D mesh given the parameters selected above.

## Output panel

Specify the working directory where the inversion occurs. For gravity and magnetic data inversion, it is **highly recommended** to chose a location on a **solid-state drive (SSD)** to fully take advantage of the `Dask` parallelization.

In [26]:
app.output_panel

**Write inpute**: Click to write the input parameters to a `*.json` file

**Run SimPEG**: Launch the inversion routine. Results will be written directly to the target `geoh5` project.

### Command line execution

Alternatively, the inversion can be launched on a different computer from command line:

`activate geoapps`

`python -m geoapps.pf_inversion [Name].json`

This assumes that the geoapps have been installed and that the reference `*.json` and `*.geoh5` project are accessible.

In [27]:
import matplotlib.pyplot as plt
app.figure.savefig("images/inversion_thumbnail.png", dpi=300, bbox_inches="tight")